<p style="font-size: 30px; font-weight: bold;">Thread Management</p>

# Thread pools

We do not need all the time a thread, we should use it only when it is necessary. It is impractial on most systems and could be expensive.

<img src="img/thread_pool.png" alt="Thread Pool" width="50%" style="">

Depending on the task queue, a thread is called from the pool when it is needed.

Why?
* Because I could optimize the core of my CPU only to use when necessary
* We need tendency more threads to hide the latency of communication
* In order to work with recursive tasks we need that the threads could also create a tasks in case of need.

```cpp
std::atomic_bool done;
thread_safe_queue<std::function<void()> > work_queue;
std::vector<std::jthread> threads;

void worker_thread() {
    while(!done) {
        std::function<void()> task;
        if(work_queue.try_pop(task))
            task();
        else
            std::this_thread::yield();
    }
}
```

```cpp
thread_pool(): done(false) {
    auto thread_count = std::thread::hardware_concurrency();
    try {
        for(auto i=0; i<thread_count; ++i) {
            threads.push_back(
            std::jthread(&thread_pool::worker_thread, this));
        }
    }
    catch(...) {
        done=true;
        throw;
    }
}

~thread_pool() {
    done = true;
}

template<typename FunctionType>
void submit(FunctionType f) {
    work_queue.push(std::function<void()>(f));
}
```

## Order of destruction

```cpp
std::atomic_bool done;
thread_safe_queue<std::function<void()> > work_queue;
std::vector<std::jthread> threads;
```

# Waiting for task submitted to a pool

If the number of blocks is small, the overhead to use the threads could lead to a bad performance

* How can a user wait for tasks to finish?
* Waiting has to be implemented manually using standard techniques
    * Condition variables
    * Futures
* Desirable to have submit() return a handle that wraps the use of condition variables or futures
* Idea – enhance implementation of task pool using packaged tasks
    * Problem: packaged tasks not copyable, prevents use of std::function
    * Requires customized wrapper

## Use case - parallel accumulate

* Parallel version of `std::accumulate`
* Computes the sum of the given value init and the elements in the range `[first, last)`

## Custom Function Wrapper

<img src="img/mgmt_wrapper.png" alt="Function Wrapper for Thread Management" width="80%" style="margin: 0 auto;">

<img src="img/mgmt_wrapper_2.png" alt="Function Wrapper for Thread Management" width="80%" style="margin: 0 auto;">

<img src="img/mgmt_wrapper_3.png" alt="Function Wrapper for Thread Management" width="80%" style="margin: 0 auto;">

<img src="img/mgmt_wrapper_4.png" alt="Function Wrapper for Thread Management" width="80%" style="margin: 0 auto;">

<img src="img/mgmt_wrapper_5.png" alt="Function Wrapper for Thread Management" width="80%" style="margin: 0 auto;">

<img src="img/mgmt_wrapper_6.png" alt="Function Wrapper for Thread Management" width="80%" style="margin: 0 auto;">

<img src="img/mgmt_wrapper_7.png" alt="Function Wrapper for Thread Management" width="80%" style="margin: 0 auto;">